## Defining the problem

**What is Writer’s Block?**
- Defined as “the inability to begin or continue writing for reasons other than a lack of basic skill or commitment.”
- Personal Experience: Can be very frustrating, caused me sleepless nights when I was affected by it. Can also happen to artists.
- Put plainly, it is a sudden lack of motivation which results in writers being unable to continue their work.

**Storyboarding, and how writer’s block has a negative effect:**
- Crucial step where creators plan the visual flow of the story, depicting scenes and panel layouts.
- Storyboarding is the bridge between a written script and the final artwork.
- Writer's block can extend to storyboarding, where creators struggle to visualize the scenes and transitions

**Writer’s Block is prevalent in the manga industry:**
- Berserk: Kentaro Miura was known for taking hiatuses due to creative challenges leading to long waits.
- Evangelion: Yoshiyuki Sadamoto took multiple hiatuses which were attributed to creative challenges.
- Hunter X Hunter: Yoshihiro Togashi, experienced writer's block, leading to year-long hiatuses.

**Creators’ relentless workload is worsening the issue:**
- There are many manga creators who end up suffering from their work
- Creators work long hours affecting physical or mental health or suffer direct abuse from the system.
- Leads to a loss of motivation, inability to produce new ideas, and worsens the problem of writer’s block.

**How is AI helping with writer's block?**
- ChatGPT is helping to write next instalment of manga hit One Piece as author runs into writer’s block.
- “Cyberpunk: Peach John” is the world’s first complete AI manga work.

We thus arrive at the following problem statement:

## Problem Statement
“Can we build AI Models that address the problem of writer’s block in the manga industry by helping with ideation and storyboarding?”

### Contents:
- [Imports](#imports)
- [Defining the functions](#defining-the-functions-necessary-to-build-the-generator)
- [Generating images](#using-the-functions-to-generate-the-image)
- [Model evaluation](#evaluation)
- [Final conclusions](#conclusions)

### Imports

In [10]:
import json
from PIL import Image, ImageDraw, ImageFont
import re
from langchain.chat_models import ChatOpenAI
from langchain.prompts.chat import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
)
import io
import os
import warnings
import random
from stability_sdk import client
import stability_sdk.interfaces.gooseai.generation.generation_pb2 as generation
import numpy as np
from skimage.metrics import structural_similarity as ssim
from skimage.metrics import peak_signal_noise_ratio as psnr

c:\Users\gauta\anaconda3\Lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated
  "class": algorithms.Blowfish,


This block imports various Python libraries and modules required for the script's functionality. Here's an explanation of some key imports: <br>

**json:** Used for working with JSON data.<br>
**PIL (Python Imaging Library):** Used for image processing, including creating, editing, and saving images.<br>
**re:** Used for working with regular expressions.<br>
**ChatOpenAI, ChatPromptTemplate, and HumanMessagePromptTemplate** are imported from custom modules <br>
**io:** Used for input and output operations, particularly working with binary data.<br>
**os:** Provides functions for interacting with the operating system, such as setting environment variables.<br>
**warnings:** Used to handle warnings in the code.<br>
**random:** Enables generating random numbers.<br>
**client and generation:** Modules related to the Stability and GooseAI APIs.<br>

## Defining the functions necessary to build the generator

In [2]:
def add_text_to_panel(text, panel_image):
    text_image = generate_text_image(text)
    result_image = Image.new('RGB', (panel_image.width, panel_image.height + text_image.height))
    result_image.paste(panel_image, (0, 0))
    result_image.paste(text_image, (0, panel_image.height))
    return result_image

def generate_text_image(text):
    # Define image dimensions
    width = 1024
    height = 128

    # Create a white background image
    image = Image.new('RGB', (width, height), color='white')

    # Create a drawing context
    draw = ImageDraw.Draw(image)

    # Choose a font
    font = ImageFont.truetype(font="manga-font.ttf", size=15)

    # Calculate text size
    text_width, text_height = draw.textsize(text, font=font)

    # Calculate the position to center the text horizontally and vertically
    x = (width - text_width) // 2
    y = (height - text_height) // 2

    # Define text color (black in this example)
    text_color = (0, 0, 0)

    # Add text to the image
    draw.text((x, y), text, fill=text_color, font=font)

    return image

### add_text_to_panel Function
The add_text_to_panel function takes two parameters: text and panel_image. Its purpose is to add text to the bottom of an image, effectively combining an image and textual content. This is commonly used for creating comic panels with dialogues or captions. <br>

Parameters:
- text: The text that you want to add to the image.
- panel_image: The image to which the text should be added. <br>

Functionality:
- text_image = generate_text_image(text): This function internally calls the generate_text_image function to create an image containing the specified text.
- result_image = Image.new('RGB', (panel_image.width, panel_image.height + text_image.height)): It creates a new image with the same width as the panel_image but a height that accommodates the original image and the text image.
- result_image.paste(panel_image, (0, 0)): It pastes the original panel_image at the top of the new result_image.
- result_image.paste(text_image, (0, panel_image.height)): It pastes the text_image at the bottom of the result_image.
- Finally, it returns the result_image, which is an image with text added to the bottom. <br>

### generate_text_image Function
The generate_text_image function creates an image with a specified text placed at the center of a white background. This is useful for generating images with text content, such as dialogues or captions. <br>

Parameters:
- text: The text that you want to appear in the image. <br>

Functionality:
- It defines the dimensions of the image, setting width and height for the white background.
- It creates a new image with a white background of the specified dimensions.
- It sets up a drawing context for the image using the Pillow library.
- It selects a font and font size for the text.
- It calculates the size of the text using the selected font.
- It determines the position to center the text horizontally and vertically within the image.
- It defines the text color (usually black in this example).
- It adds the specified text to the image, ensuring that it is centered.
- The function returns the generated image with the specified text on a white background. <br>

These two functions work in conjunction to create images with text content, which is a common requirement for generating comic panels or similar visual content.

In [3]:
# Function to resize and add a black border to an image
def resize_and_add_border(image, target_size, border_size):
    resized_image = Image.new("RGB", target_size, "black")
    resized_image.paste(image, ((target_size[0] - image.width) // 2, (target_size[1] - image.height) // 2))
    return resized_image

def create_strip(images):
    # Desired grid size
    columns, rows = 2, 3

    # Calculate the size of the output image
    output_width = columns * images[0].width + (columns - 1) * 10  # 10 is the black border width
    output_height = rows * images[0].height + (rows - 1) * 10  # 10 is the black border width

    # Create a new image with the calculated size
    result_image = Image.new("RGB", (output_width, output_height), "white")

    # Combine images into a grid with black borders
    for i, img in enumerate(images):
        x = (i % columns) * (img.width + 10)  # 10 is the black border width
        y = (i // columns) * (img.height + 10)  # 10 is the black border width

        resized_img = resize_and_add_border(img, (images[0].width, images[0].height), 10)
        result_image.paste(resized_img, (x, y))

    return result_image.resize((1024, 1536))

Here's a breakdown of what the code does:

**resize_and_add_border function:**

- This function takes three parameters: image (an input image), target_size (a tuple representing the desired size of the output image), and border_size (an integer representing the width of the black border to be added).
- It creates a new black image of the specified target_size.
- It pastes the input image onto the center of the black image, effectively resizing and adding a black border around the input image.
- The resulting image is returned.

**create_strip function:**

- This function takes a list of images as its input, assumed to be stored in the images variable.
- It defines the desired grid layout for the output image using the columns and rows variables, which determine how many images are placed horizontally and vertically in the grid.
- It calculates the size of the output image (output_width and output_height) based on the number of columns, rows, and the width of the black borders (10 pixels).
- It creates a new white image with the calculated size.
- It then iterates over the input images and arranges them in the grid with black borders:
- The x and y variables are calculated to determine the position of each image in the grid, considering the image's width, height, and the 10-pixel black border between them.
- The resize_and_add_border function is called to resize the input image to the size of the first image (images[0]) and add a 10-pixel black border.
- The resized and bordered image is pasted onto the result image at the calculated position (x, y).

Finally, the result image is resized to a fixed size of 1024x1536 pixels, and the resulting image is returned.

In [4]:
os.environ['OPENAI_API_KEY'] = 'your-api-key'

The code sets the environment variable named 'OPENAI_API_KEY' with a specific API key value.

In [5]:
template = """
You are a manga artist.

Your task is to take a brief storyline and divide it into six distinct manga panels. 
Each manga panel requires a detailed description, including the characters and the panel's background. 
Character descriptions should be precise, and avoid using character names within the panel descriptions. 
You must also provide concise text for each panel, consisting of no more than two short sentences that start with the character's name.

Example Input:
Characters: Lily is a cheerful girl with a ribbon in her hair. Max is a mischievous boy with a backpack.
Lily and Max set off on a thrilling journey, venturing deep into the mysterious cave of wonders.

Example Output:

# Panel 1
description: Two companions, a girl with a ribbon in her hair and a mischievous boy with a backpack, stand at the cave entrance, surrounded by eerie stalactites.

text:
```
Lily: The cave looks so mysterious!
Max: Let's explore and see what's inside.
```
# end

Short Scenario:
{scenario}

Split the scenario in 6 parts:
"
"""

def generate_panels(scenario):
    model = ChatOpenAI(model_name='gpt-3.5-turbo')

    human_message_prompt = HumanMessagePromptTemplate.from_template(template)

    chat_prompt = ChatPromptTemplate.from_messages([human_message_prompt])

    chat_prompt.format_messages(scenario=scenario)

    result = model(chat_prompt.format_messages(scenario=scenario))

    print(result.content)

    return extract_panel_info(result.content)

def extract_panel_info(text):
    panel_info_list = []
    panel_blocks = text.split('# Panel')

    for block in panel_blocks:
        if block.strip() != '':
            panel_info = {}
            
            # Extracting panel number
            panel_number = re.search(r'\d+', block)
            if panel_number is not None:
                panel_info['number'] = panel_number.group()
            
            # Extracting panel description
            panel_description = re.search(r'description: (.+)', block)
            if panel_description is not None:
                panel_info['description'] = panel_description.group(1)
            
            # Extracting panel text
            panel_text = re.search(r'text:\n```\n(.+)\n```', block, re.DOTALL)
            if panel_text is not None:
                panel_info['text'] = panel_text.group(1)
            
            panel_info_list.append(panel_info)
    return panel_info_list

Here's a breakdown of the code and what it does:

The template variable contains a template that sets the context for the manga story generation. It describes the task and requirements for dividing a storyline into six manga panels, including character descriptions, backgrounds, and character dialogues.

The generate_panels function takes a scenario as input, which represents the storyline. It uses the OpenAI GPT-3.5 Turbo model to generate the manga panels based on the scenario.

Within the generate_panels function:

- It initializes a GPT-3.5 Turbo model from the chat-openai library.
- It defines a human message prompt template using the provided template.
- It creates a chat prompt template and formats it with the given scenario.
- It uses the GPT-3.5 Turbo model to generate content based on the chat prompt and scenario.
- It prints the generated result.
- The extract_panel_info function is used to extract information about each manga panel from the generated text:

- It splits the generated text into individual panel blocks using the # Panel delimiter.
- For each panel block, it extracts panel number, panel description, and panel text using regular expressions.
- The extracted information is stored in a list of dictionaries, where each dictionary represents a manga panel with its number, description, and text.
- The function returns a list of panel information.

In [6]:
os.environ['STABILITY_HOST'] = 'grpc.stability.ai:443'

seed = random.randint(0, 1000000000)

# Set up our connection to the API.
stability_api = client.StabilityInference(
    key="your-api-key",
    verbose=True,
    engine="stable-diffusion-xl-1024-v1-0"
)

def text_to_image(prompt):
    # Set up our initial generation parameters.
    answers = stability_api.generate(
        # prompt="rocket ship launching from forest with flower garden under a blue sky, masterful, ghibli",
        prompt=prompt,
        seed=seed, # If a seed is provided, the resulting generated image will be deterministic.
                        # What this means is that as long as all generation parameters remain the same, you can always recall the same image simply by generating it again.
                        # Note: This isn't quite the case for CLIP Guided generations, which we tackle in the CLIP Guidance documentation.
        steps=30, # Amount of inference steps performed on image generation. Defaults to 30.
        cfg_scale=8.0, # Influences how strongly your generation is guided to match your prompt.
                    # Setting this value higher increases the strength in which it tries to match your prompt.
                    # Defaults to 7.0 if not specified.
        width=1024, # Generation width, defaults to 1024 if not included.
        height=1024, # Generation height, defaults to 1024 if not included.
        sampler=generation.SAMPLER_K_DPMPP_2M # Choose which sampler we want to denoise our generation with.
                                                    # Defaults to k_dpmpp_2m if not specified. Clip Guidance only supports ancestral samplers.
                                                    # (Available Samplers: ddim, plms, k_euler, k_euler_ancestral, k_heun, k_dpm_2, k_dpm_2_ancestral, k_dpmpp_2s_ancestral, k_lms, k_dpmpp_2m, k_dpmpp_sde)
    )

    # Set up our warning to print to the console if the adult content classifier is tripped.
    # If adult content classifier is not tripped, display generated image.
    for resp in answers:
        for artifact in resp.artifacts:
            if artifact.finish_reason == generation.FILTER:
                warnings.warn(
                    "Your request activated the API's safety filters and could not be processed."
                    "Please modify the prompt and try again.")
            if artifact.type == generation.ARTIFACT_IMAGE:
                global img
                img = Image.open(io.BytesIO(artifact.binary))
                return img 

def edit_image(input_image_path, prompt, output_image_name):
    img = Image.open(input_image_path)

    # Set up our initial generation parameters.
    answers = stability_api.generate(
        # prompt="crayon drawing of rocket ship launching from forest",
        prompt=prompt,
        init_image=img, # Assign our previously generated img as our Initial Image for transformation.
        start_schedule=0.7, # Set the strength of our prompt in relation to our initial image.
        seed=123463446, # If attempting to transform an image that was previously generated with our API,
                        # initial images benefit from having their own distinct seed rather than using the seed of the original image generation.
        steps=50, # Amount of inference steps performed on image generation. Defaults to 30.
        cfg_scale=8.5, # Influences how strongly your generation is guided to match your prompt.
                    # Setting this value higher increases the strength in which it tries to match your prompt.
                    # Defaults to 7.0 if not specified.
        width=512, # Generation width, defaults to 512 if not included.
        height=512, # Generation height, defaults to 512 if not included.
        sampler=generation.SAMPLER_K_DPMPP_2M # Choose which sampler we want to denoise our generation with.
                                                    # Defaults to k_dpmpp_2m if not specified. Clip Guidance only supports ancestral samplers.
                                                    # (Available Samplers: ddim, plms, k_euler, k_euler_ancestral, k_heun, k_dpm_2, k_dpm_2_ancestral, k_dpmpp_2s_ancestral, k_lms, k_dpmpp_2m, k_dpmpp_sde)
    )

    # Set up our warning to print to the console if the adult content classifier is tripped.
    # If adult content classifier is not tripped, save generated image.
    for resp in answers:
        for artifact in resp.artifacts:
            if artifact.finish_reason == generation.FILTER:
                warnings.warn(
                    "Your request activated the API's safety filters and could not be processed."
                    "Please modify the prompt and try again.")
            if artifact.type == generation.ARTIFACT_IMAGE:
                global img2
                img2 = Image.open(io.BytesIO(artifact.binary))
                img2.save(output_image_name + ".png") # Save our generated image with its seed number as the filename and the img2img suffix so that we know this is our transformed image.

The code involves two main functions: text_to_image and edit_image. The code utilizes the Stability API to generate and edit images based on provided prompts and configurations. Here's a breakdown of what the code does:

Environment Variable Setup:

- It sets an environment variable named 'STABILITY_HOST' with the value 'grpc.stability.ai:443'. This specifies the host and port for the Stability API.

Random Seed Generation:

- It generates a random seed using the random.randint function. This seed can be used to make image generation deterministic.

API Setup:

- It sets up a connection to the Stability API using the client.StabilityInference class. This includes specifying the API key, the engine and enabling verbosity for debugging.

text_to_image Function:

- This function generates an image based on a given prompt:
- It calls the Stability API's generate method with parameters like the prompt, random seed, and other generation settings.
- It checks if the generated image contains adult content and issues a warning if it does.
- If no issues are detected, it returns the generated image.

edit_image Function:

- This function takes an existing image and modifies it based on a new prompt:
- It opens an input image from a specified file path.
- It calls the Stability API's generate method with parameters like the prompt, the input image as the initial image, and other generation settings.
- It checks if the generated image contains adult content and issues a warning if it does.
- If no issues are detected, it saves the modified image with a specified output file name.

### Using the functions to generate the image

In [8]:
SCENARIO = """
Characters: Aiko Nakamura: Aiko is a 17-year-old high school student with long, midnight-black hair and striking amethyst eyes. She's introverted, passionate about astronomy, and often seen with her telescope. She has a quiet, mysterious aura and a tendency to stargaze at night.
Hiroshi Tanaka: Hiroshi is an 18-year-old classmate of Aiko's, known for his outgoing and carefree personality. He has messy, sun-kissed hair and deep green eyes. He's a soccer player and the school's popular guy, often surrounded by friends.
Aiko Nakamura, a girl who lives in the world of stars and constellations, finds herself secretly in love with Hiroshi Tanaka, the school heartthrob who is always surrounded by adoring fans. The story begins on a clear, moonlit night. Aiko is on her balcony, observing the constellations with her telescope, as she does every night. Unexpectedly, she spots Hiroshi on his own balcony, staring up at the same moon and stars. Aiko can't help but be captivated by the sight of him lost in thought, his usually charismatic demeanor replaced by something more vulnerable. She realizes that Hiroshi has a hidden love for astronomy as well, a shared passion they've never discussed. Aiko takes a deep breath and gathers her courage, deciding to approach Hiroshi. She sends him a message, asking if he'd like to join her for some stargazing. At first, Hiroshi is taken aback by the message from Aiko, the enigmatic girl from their school who he's admired from afar. However, his curiosity and love for the stars compel him to accept her invitation. As they share the moonlit balcony and gaze at the night sky, Aiko and Hiroshi bond over their shared passion for astronomy. They discuss constellations, the beauty of the cosmos, and their dreams. With each passing moment, they become closer, and a romantic connection begins to bloom beneath the stars.
"""
STYLE = "manga comic, greyscale"

# Generate panels
print(f"Generate panels with style '{STYLE}' for this scenario: \n {SCENARIO}")
panels = generate_panels(SCENARIO)

# Save generated panels to a JSON file
with open('output/panels.json', 'w') as outfile:
    json.dump(panels, outfile)

# Initialize a list to store panel images
panel_images = []

# Generate images for each panel and add text
for panel in panels:
    panel_prompt = panel["description"] + ", cartoon box, " + STYLE
    print(f"Generate panel {panel['number']} with prompt: {panel_prompt}")
    panel_image = text_to_image(panel_prompt)
    panel_image_with_text = add_text_to_panel(panel["text"], panel_image)
    panel_image_with_text.save(f"output/panel-{panel['number']}.png")
    panel_images.append(panel_image_with_text)

# Create a strip from the panel images and save it
create_strip(panel_images).save("output/strip.png")

Generate panels with style 'manga comic, greyscale' for this scenario: 
 
Characters: Aiko Nakamura: Aiko is a 17-year-old high school student with long, midnight-black hair and striking amethyst eyes. She's introverted, passionate about astronomy, and often seen with her telescope. She has a quiet, mysterious aura and a tendency to stargaze at night.
Hiroshi Tanaka: Hiroshi is an 18-year-old classmate of Aiko's, known for his outgoing and carefree personality. He has messy, sun-kissed hair and deep green eyes. He's a soccer player and the school's popular guy, often surrounded by friends.
Aiko Nakamura, a girl who lives in the world of stars and constellations, finds herself secretly in love with Hiroshi Tanaka, the school heartthrob who is always surrounded by adoring fans. The story begins on a clear, moonlit night. Aiko is on her balcony, observing the constellations with her telescope, as she does every night. Unexpectedly, she spots Hiroshi on his own balcony, staring up at the s

C:\Users\gauta\AppData\Local\Temp\ipykernel_4340\24759753.py:23: DeprecationWarning: textsize is deprecated and will be removed in Pillow 10 (2023-07-01). Use textbbox or textlength instead.
  text_width, text_height = draw.textsize(text, font=font)


Generate panel 2 with prompt: Hiroshi Tanaka, an 18-year-old boy with messy, sun-kissed hair and deep green eyes, stands on his own balcony, gazing up at the moon and stars., cartoon box, manga comic, greyscale


C:\Users\gauta\AppData\Local\Temp\ipykernel_4340\24759753.py:23: DeprecationWarning: textsize is deprecated and will be removed in Pillow 10 (2023-07-01). Use textbbox or textlength instead.
  text_width, text_height = draw.textsize(text, font=font)


Generate panel 3 with prompt: Aiko, captivated by the sight of Hiroshi lost in thought, gathers her courage to approach him., cartoon box, manga comic, greyscale


...

C:\Users\gauta\AppData\Local\Temp\ipykernel_4340\24759753.py:23: DeprecationWarning: textsize is deprecated and will be removed in Pillow 10 (2023-07-01). Use textbbox or textlength instead.
  text_width, text_height = draw.textsize(text, font=font)


Generate panel 5 with prompt: Aiko and Hiroshi share the moonlit balcony, engrossed in their conversation about constellations and the beauty of the cosmos., cartoon box, manga comic, greyscale


C:\Users\gauta\AppData\Local\Temp\ipykernel_4340\24759753.py:23: DeprecationWarning: textsize is deprecated and will be removed in Pillow 10 (2023-07-01). Use textbbox or textlength instead.
  text_width, text_height = draw.textsize(text, font=font)


Generate panel 6 with prompt: Aiko and Hiroshi's connection deepens as they continue to stargaze, their hearts opening up beneath the starry night., cartoon box, manga comic, greyscale


C:\Users\gauta\AppData\Local\Temp\ipykernel_4340\24759753.py:23: DeprecationWarning: textsize is deprecated and will be removed in Pillow 10 (2023-07-01). Use textbbox or textlength instead.
  text_width, text_height = draw.textsize(text, font=font)


### Evaluation
**SSIM (Structural Similarity Index):**

SSIM, which stands for Structural Similarity Index, is a metric used to assess the structural similarity between two images.
It quantifies how similar two images are in terms of their structural patterns, luminance, and contrast.
SSIM values range from -1 to 1, where 1 indicates that the two images are identical in terms of structure and appearance.
SSIM is particularly useful for comparing and evaluating the similarity between generated images. Higher SSIM values suggest that the images have similar structural patterns and are visually more alike. Lower SSIM values indicate differences in structure.

**PSNR (Peak Signal-to-Noise Ratio):**

PSNR, or Peak Signal-to-Noise Ratio, is a metric used to measure the quality of an image by comparing it to a reference or original image.
It calculates the ratio of the peak signal (the highest possible pixel value) to the root mean square error (RMSE) between the original and generated images.
PSNR is expressed in decibels (dB), and higher PSNR values indicate higher image quality. A higher PSNR means that the generated image closely resembles the original image.
PSNR can also be used to compare the quality of generated images relative to each other. It helps assess which generated images have better quality in a relative sense.
When evaluating generated images, both SSIM and PSNR are valuable tools. SSIM provides insights into how similar the structures and patterns are between images, while PSNR quantifies image quality.

In [11]:
# Convert PIL images to NumPy arrays
panel_images_np = [np.array(panel_image) for panel_image in panel_images]

# Initialize lists to store SSIM and PSNR scores for each panel
ssim_scores = []
psnr_scores = []

# Specify a smaller win_size
win_size = 3  # You can adjust this value as needed

# Compare generated panels against each other
for i, panel_image in enumerate(panel_images_np):
    for j, reference_image in enumerate(panel_images_np):
        if i != j:
            ssim_score = ssim(panel_image, reference_image, multichannel=True, win_size=win_size)
            psnr_score = psnr(panel_image, reference_image)
            ssim_scores.append(ssim_score)
            psnr_scores.append(psnr_score)

# Print the SSIM and PSNR scores for each panel
panel_count = len(panel_images)
for i in range(panel_count):
    for j in range(panel_count):
        if i != j:
            similarity_index = i * (panel_count - 1) + j
            print(f"Panel {i+1} vs. Panel {j+1}: SSIM = {ssim_scores[similarity_index]:.2f}, PSNR = {psnr_scores[similarity_index]:.2f}")

Panel 1 vs. Panel 2: SSIM = 0.46, PSNR = 8.89
Panel 1 vs. Panel 3: SSIM = 0.47, PSNR = 8.13
Panel 1 vs. Panel 4: SSIM = 0.52, PSNR = 11.74
Panel 1 vs. Panel 5: SSIM = 0.48, PSNR = 10.37
Panel 1 vs. Panel 6: SSIM = 0.57, PSNR = 12.71
Panel 2 vs. Panel 1: SSIM = 0.57, PSNR = 12.71
Panel 2 vs. Panel 3: SSIM = 0.48, PSNR = 8.41
Panel 2 vs. Panel 4: SSIM = 0.54, PSNR = 11.76
Panel 2 vs. Panel 5: SSIM = 0.49, PSNR = 10.71
Panel 2 vs. Panel 6: SSIM = 0.46, PSNR = 8.89
Panel 3 vs. Panel 1: SSIM = 0.46, PSNR = 8.89
Panel 3 vs. Panel 2: SSIM = 0.47, PSNR = 9.31
Panel 3 vs. Panel 4: SSIM = 0.46, PSNR = 9.87
Panel 3 vs. Panel 5: SSIM = 0.51, PSNR = 10.60
Panel 3 vs. Panel 6: SSIM = 0.47, PSNR = 8.13
Panel 4 vs. Panel 1: SSIM = 0.47, PSNR = 8.13
Panel 4 vs. Panel 2: SSIM = 0.48, PSNR = 8.41
Panel 4 vs. Panel 3: SSIM = 0.60, PSNR = 12.19
Panel 4 vs. Panel 5: SSIM = 0.50, PSNR = 9.50
Panel 4 vs. Panel 6: SSIM = 0.52, PSNR = 11.74
Panel 5 vs. Panel 1: SSIM = 0.52, PSNR = 11.74
Panel 5 vs. Panel 2: SSI

### Interpretation
**SSIM (Structural Similarity Index):**

- The SSIM scores for the generated panels vary from approximately 0.46 to 0.60.
- These scores indicate moderate to strong structural similarity between different panels.

**PSNR (Peak Signal-to-Noise Ratio):**

- The PSNR scores for your generated panels range from around 8.13 to 12.71.
- These scores suggest a range of image quality, with some panels having lower image quality and others having higher image quality.

Overall, the SSIM and PSNR scores show that there is a diversity in the quality and structural similarity of the generated panels. Some panels exhibit stronger structural similarity and higher image quality, while others have lower scores, indicating differences in visual patterns and image quality.

### Conclusions
With this, we have created an image generator to generate a manga strip based on a given scenario. We can now finish the project and answer the problem statement.
We have created a way to automated ideation and storyboarding using generative AI tools. These tools can provide several benefits to manga writers and help them overcome writer's block:
- **Inspiration and Creativity Boost:** The tool can provide fresh and creative ideas, helping manga writers overcome creative blocks. It can offer novel scenarios, character interactions, and plot twists, sparking new inspiration.
- **Overcoming Writer's Block:** Our tool can break the writer's block by providing a starting point. Writers can modify, expand upon, or adapt these prompts to fit their narrative vision.
- **Reducing Writer's Stress:** By taking on some of the ideation workload, the tool can reduce the stress and pressure associated with coming up with new story ideas and storyboards on a consistent basis.
- **Collaboration:** The generated panels produced by our tool can serve as a starting point for collaboration between writers and artists. Writers can present the generated concepts to artists, who can then refine and visualize them. This is especially a boon to people who like to write, but cannot draw.